In [46]:
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import nltk
nltk.download("stopwords")
#--------#
from string import punctuation

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    words = re.split(r'[^а-яА-Я]', text) # разбиваем текст на слова
    tokens = list()
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS not in ['NPRO', 'PREP', 'CONJ', 'PRCL', 'INTJ', 'NUMR']:
            tokens.append(p.normal_form)
    tokens = [token for token in tokens if token not in russian_stopwords\
                and token != " " \
                and token.strip() not in punctuation]
    company_tokens_text_len = len(tokens)

    return " ".join(tokens), company_tokens_text_len

def preprocess_text_in_list(list_):
    prepr_list = []
    for comment in list_:
        prepr_comment = preprocess_text(comment)
        prepr_list.append(prepr_comment)
    return prepr_list

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     Удаленный хост принудительно разорвал существующее
[nltk_data]     подключение>


In [47]:
import os

groups_files = os.listdir('Collected')

groups_files

['Бонч_мемы_all_posts_31_07_2023.xlsx',
 'Бонч_основная_all_posts_31_07_2023.xlsx',
 'Бонч_подслушано_all_posts_31_07_2023.xlsx',
 'Бонч_поступление_all_posts_31_07_2023.xlsx',
 'ВШЭ_мемы_all_posts_31_07_2023.xlsx',
 'ВШЭ_основная_all_posts_31_07_2023.xlsx',
 'ВШЭ_поступление_all_posts_31_07_2023.xlsx',
 'ИТМО_мемы_all_posts_31_07_2023.xlsx',
 'ИТМО_основная_all_posts_31_07_2023.xlsx',
 'ИТМО_подслушано_all_posts_31_07_2023.xlsx',
 'ИТМО_поступление_all_posts_31_07_2023 — копия.xlsx',
 'Мечникова_мемы_all_posts_31_07_2023.xlsx',
 'Мечникова_основная_all_posts_31_07_2023.xlsx',
 'Мечникова_подслушано_all_posts_31_07_2023.xlsx',
 'Мечникова_профком_all_posts_31_07_2023.xlsx',
 'Политех_мемы_all_posts_31_07_2023.xlsx',
 'Политех_основная_all_posts_31_07_2023.xlsx',
 'Политех_подслушано_all_posts_31_07_2023.xlsx',
 'Политех_поступление_all_posts_31_07_2023.xlsx']

In [48]:
import pandas as pd
from ast import literal_eval #used for extracting comments (lists stored as strings)

os.makedirs('Preprocessed', exist_ok = True)
for group in groups_files:
    group_df = pd.read_excel(f'Collected/{group}')

    text_group_df = group_df[['Текст_поста', "Текст_комментариев"]]

    text_group_df_preprocessed = text_group_df['Текст_поста'].astype(str).apply(preprocess_text)
    text_group_df_preprocessed = pd.DataFrame(list(text_group_df_preprocessed), columns = ['text_tokens', 'text_length'])
    prepr_df = pd.concat([group_df, text_group_df_preprocessed], axis=1, join='inner')

    prepr_df['preprocessed_comments'] = text_group_df["Текст_комментариев"].apply(literal_eval).apply(preprocess_text_in_list)

    prepr_df.to_csv(f'Preprocessed/{group[:-5]}.csv')

SyntaxError: EOL while scanning string literal (<unknown>, line 1)